In [1]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, Set, Binary, SolverFactory, maximize
import random
import pandas as pd

In [2]:
# # Sample SKU data. Each SKU is characterized by its net sales, OTIF, turnover, NPS, and a family identifier.
# sku_data = {
#     'sku1': {'net_sales': 100, 'OTIF': 0.95, 'turnover': 200, 'NPS': 8, 'family': 'A'},
#     'sku2': {'net_sales': 150, 'OTIF': 0.90, 'turnover': 300, 'NPS': 7, 'family': 'B'},
#     'sku3': {'net_sales': 120, 'OTIF': 0.98, 'turnover': 250, 'NPS': 9, 'family': 'C'},
#     'sku4': {'net_sales': 130, 'OTIF': 0.92, 'turnover': 220, 'NPS': 8, 'family': 'D'},
#     # Add additional SKU records as needed
# }
# pd.DataFrame(sku_data)

In [37]:
otif_gen = lambda : round(random.random() * 100, 2)
sku_gen = lambda : 'sku' + str(random.randint(0, 100))
int_gen = lambda x: random.randint(0, x)
float_gen = lambda x: random.random() * x
family_gen = lambda : random.choice(['A', 'B', 'C', 'D', 'E'])


sku_data = {
    'sku' + str(i): {
        "net_sales": float_gen(10000),
        "OTIF": otif_gen(),
        "turnover": int_gen(100),
        "NPS": int_gen(10),
        "family": int_gen(100)
    } for i in range(1000000)
}

pd.DataFrame(sku_data)

,sku0,sku1,sku2,sku3,sku4,sku5,sku6,sku7,sku8,sku9,...,sku999990,sku999991,sku999992,sku999993,sku999994,sku999995,sku999996,sku999997,sku999998,sku999999
net_sales,8361.801297,2714.331537,5413.508445,5842.772711,5930.316902,6801.114378,3959.010665,8662.473778,8942.06792,335.55227,...,4048.929072,8376.617312,8183.137938,7448.797521,1684.577232,9373.247408,3190.620201,9658.05396,5794.263241,7709.23951
OTIF,79.360000,24.080000,63.110000,56.840000,0.170000,12.850000,33.500000,7.560000,2.96000,62.13000,...,91.490000,71.460000,59.230000,67.260000,15.790000,20.510000,63.360000,79.00000,2.300000,94.30000
turnover,76.000000,13.000000,2.000000,64.000000,20.000000,94.000000,16.000000,51.000000,16.00000,43.00000,...,73.000000,21.000000,62.000000,62.000000,90.000000,100.000000,51.000000,93.00000,15.000000,31.00000
NPS,2.000000,1.000000,9.000000,5.000000,10.000000,1.000000,10.000000,0.000000,3.00000,9.00000,...,3.000000,10.000000,8.000000,5.000000,0.000000,1.000000,7.000000,8.00000,8.000000,2.00000
family,12.000000,20.000000,79.000000,55.000000,19.000000,68.000000,41.000000,40.000000,39.00000,64.00000,...,98.000000,84.000000,3.000000,4.000000,88.000000,6.000000,44.000000,87.00000,59.000000,68.00000


In [43]:
# Parameters
max_skus = 100        # Maximum number of SKUs that can be placed at the checkout shelf.
OTIF_min = 0.4       # Minimum average OTIF required.
turnover_min = 40    # Minimum total turnover required.
NPS_min = 7         # Minimum average NPS required.

# Create Pyomo model
model = ConcreteModel()

# Define a set of SKUs
model.SKUs = Set(initialize=sku_data.keys())

# Create dictionaries for the parameters
net_sales = {sku: sku_data[sku]['net_sales'] for sku in sku_data}
OTIF = {sku: sku_data[sku]['OTIF'] for sku in sku_data}
turnover = {sku: sku_data[sku]['turnover'] for sku in sku_data}
NPS = {sku: sku_data[sku]['NPS'] for sku in sku_data}
family = {sku: sku_data[sku]['family'] for sku in sku_data}

# Decision variables: x[sku] = 1 if the SKU is selected, 0 otherwise.
model.x = Var(model.SKUs, domain=Binary)

# Objective: maximize total net sales from the selected SKUs.
# expr=sum(net_sales[sku] * model.x[sku] for sku in model.SKUs),
alpha = 10
beta = 10000
model.obj = Objective(
    expr = alpha * sum(net_sales[sku] * model.x[sku] for sku in model.SKUs) +
           beta * sum(OTIF[sku] * model.x[sku] for sku in model.SKUs),
    sense=maximize
)

# Constraint 1: Select at most 'max_skus' SKUs.
model.maxSKUConstraint = Constraint(
    expr=sum(model.x[sku] for sku in model.SKUs) <= max_skus
)

# Constraint 2: Ensure the average OTIF of selected SKUs is at least OTIF_min.
# This is modeled as: sum((OTIF_i - OTIF_min) * x_i) >= 0.
model.OTIFConstraint = Constraint(
    expr=sum((OTIF[sku] - OTIF_min) * model.x[sku] for sku in model.SKUs) >= 0
)

# Constraint 3: Ensure the total turnover from selected SKUs is at least turnover_min.
model.turnoverConstraint = Constraint(
    expr=sum(turnover[sku] * model.x[sku] for sku in model.SKUs) >= turnover_min
)

print(f'\n\n Fabio | len(model.x) = {type(model.x)}')

# Constraint 4: Ensure the average NPS of selected SKUs is at least NPS_min.
# This is modeled similarly: sum((NPS_i - NPS_min) * x_i) >= 0.
model.NPSConstraint = Constraint(
    expr=sum((NPS[sku] - NPS_min) * model.x[sku] for sku in model.SKUs) >= 0
)

# Constraint 5: At most one SKU per product family.
# First, determine the unique families.
unique_families = set(family.values())

def family_rule(model, fam):
    return sum(model.x[sku] for sku in model.SKUs if family[sku] == fam) <= 1

model.familyConstraint = Constraint(unique_families, rule=family_rule)

# Solve the model using a free solver (e.g., CBC or GLPK)
solver = SolverFactory('cbc')  # Alternatively, use 'glpk' if CBC is unavailable.
result = solver.solve(model, tee=True)

# Output the solution
print("Status:", result.solver.status)
print("Termination Condition:", result.solver.termination_condition)

selected_skus = [sku for sku in model.SKUs if model.x[sku].value is not None and model.x[sku].value >= 0.99]
print("Selected SKUs:", selected_skus)
total_net_sales = sum(net_sales[sku] * model.x[sku].value for sku in model.SKUs if model.x[sku].value is not None)
print("Total Net Sales:", total_net_sales)



 Fabio | len(model.x) = <class 'pyomo.core.base.var.IndexedVar'>
Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Sep  3 2024 

command line - /Users/fabioyamada/miniconda3/envs/.optimization/bin/cbc -printingOptions all -import /var/folders/1j/2s04pnsd0_n4srzj9y2v33qh0000gn/T/tmpco3wc4qx.pyomo.lp -stat=1 -solve -solu /var/folders/1j/2s04pnsd0_n4srzj9y2v33qh0000gn/T/tmpco3wc4qx.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve 105 (0) rows, 1000000 (0) columns and 4899347 (0) elements
Statistics for presolved model
Original problem has 1000000 integers (1000000 of which binary)
==== 0 zero objective 1000000 different
==== absolute objective values 1000000 different
==== for integers 0 zero objective 1000000 different
==== for integers absolute objective values 1000000 different
===== end obje

In [44]:
[(sku,model.x[sku].value, net_sales[sku], OTIF[sku]) for sku in model.SKUs if model.x[sku].value != 0]

[('sku120', 1.0, 9869.665098052517, 99.82),
 ('sku17399', 1.0, 9847.66092660221, 99.96),
 ('sku23982', 1.0, 9958.22153136178, 99.57),
 ('sku44320', 1.0, 9682.526705525283, 99.83),
 ('sku65447', 1.0, 9998.113474594631, 99.99),
 ('sku72488', 1.0, 9666.907960425013, 99.79),
 ('sku76467', 1.0, 9888.90978255999, 99.76),
 ('sku79159', 1.0, 9997.450214925133, 99.87),
 ('sku79361', 1.0, 9768.038878260426, 99.84),
 ('sku85018', 1.0, 9771.6025022733, 99.89),
 ('sku105973', 1.0, 9842.386253820338, 99.85),
 ('sku110447', 1.0, 9967.824823471863, 99.8),
 ('sku117717', 1.0, 9974.026313930523, 99.85),
 ('sku124999', 1.0, 9833.517965413488, 99.97),
 ('sku142318', 1.0, 9875.448124417793, 99.66),
 ('sku169488', 1.0, 9415.880860310846, 99.83),
 ('sku179749', 1.0, 9946.058553147537, 99.44),
 ('sku204120', 1.0, 9798.141099592473, 99.81),
 ('sku210042', 1.0, 9858.987650796174, 99.68),
 ('sku215777', 1.0, 9842.060917051529, 99.79),
 ('sku219030', 1.0, 9947.07288241897, 99.84),
 ('sku240134', 1.0, 9466.9509074

In [40]:
net_sales

{'sku0': 8361.801297071617,
 'sku1': 2714.3315366282973,
 'sku2': 5413.5084452219835,
 'sku3': 5842.77271121653,
 'sku4': 5930.316901628166,
 'sku5': 6801.114377644645,
 'sku6': 3959.0106654232536,
 'sku7': 8662.473778142261,
 'sku8': 8942.067919801795,
 'sku9': 335.55226996376143,
 'sku10': 2257.838739434466,
 'sku11': 8543.389586605845,
 'sku12': 1121.492953903551,
 'sku13': 8215.620274057954,
 'sku14': 1262.2223770625708,
 'sku15': 7365.804510498677,
 'sku16': 4797.026544318001,
 'sku17': 5146.403518331803,
 'sku18': 4474.961493515426,
 'sku19': 9966.173117663695,
 'sku20': 6609.820262314521,
 'sku21': 2295.5622405282093,
 'sku22': 1426.5167920910903,
 'sku23': 7290.562560360687,
 'sku24': 4907.385537004561,
 'sku25': 9637.35566467621,
 'sku26': 5955.87091559744,
 'sku27': 2133.4294691262558,
 'sku28': 457.4898197324551,
 'sku29': 1526.0868665407313,
 'sku30': 108.18643985766263,
 'sku31': 2065.01082535669,
 'sku32': 6832.985869670422,
 'sku33': 1809.3421312390922,
 'sku34': 9438.91

In [29]:
family.values()

dict_values(['D', 'D', 'E', 'C', 'E', 'D', 'C', 'A', 'E', 'E', 'B', 'B', 'E', 'B', 'D', 'C', 'A', 'B', 'B', 'E', 'B', 'B', 'E', 'D', 'A', 'B', 'D', 'E', 'B', 'A', 'C', 'A', 'E', 'C', 'D', 'E', 'B', 'B', 'E', 'A', 'C', 'C', 'B', 'E', 'B', 'E', 'E', 'D', 'C', 'C', 'D', 'D', 'E', 'B', 'E', 'B', 'C', 'E', 'A', 'B', 'C', 'B', 'D', 'C', 'A', 'D', 'D', 'E', 'D', 'E', 'D', 'E', 'E', 'D', 'D', 'B', 'B', 'C', 'C', 'B', 'B', 'B', 'A', 'D', 'C', 'B', 'A', 'E', 'B', 'C', 'A', 'A', 'A', 'E', 'B', 'A', 'E', 'B', 'B', 'B', 'B', 'A', 'B', 'E', 'E', 'B', 'D', 'E', 'E', 'A', 'C', 'D', 'D', 'A', 'E', 'B', 'B', 'A', 'E', 'C', 'A', 'A', 'D', 'E', 'D', 'A', 'D', 'D', 'B', 'E', 'C', 'B', 'B', 'E', 'A', 'A', 'C', 'C', 'C', 'A', 'D', 'B', 'E', 'B', 'D', 'E', 'C', 'E', 'C', 'C', 'E', 'C', 'C', 'C', 'C', 'A', 'E', 'D', 'D', 'D', 'C', 'B', 'C', 'B', 'B', 'A', 'E', 'A', 'B', 'D', 'D', 'B', 'E', 'C', 'B', 'B', 'A', 'C', 'E', 'D', 'C', 'E', 'E', 'C', 'A', 'D', 'A', 'E', 'E', 'D', 'B', 'B', 'B', 'C', 'E', 'D', 'C', 'B